In [1]:
from param import ml_names, pers_names, data_name, data_path, results_path, thres_sel_feat
from ml_models import loocv_day, agg_feat_imp
from ml_models import MAE, MSE, RMSE, MAPE, R2
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir(results_path)
df_ml_input_new = pd.read_csv('ml_input.csv', index_col = 0)

In [3]:
df_ml_input_new

,participantID,phone_battery_rapids_sumdurationdischarge,phone_battery_rapids_avgconsumptionrate,phone_battery_rapids_countcharge,phone_battery_rapids_sumdurationcharge,phone_battery_rapids_countdischarge,phone_battery_rapids_maxconsumptionrate,phone_calls_rapids_missed_count,phone_calls_rapids_missed_distinctcontacts,phone_calls_rapids_missed_timefirstcall,...,neoConscientiousness,neoNeuroticism,neoExtraversion,neoAgreeableness,neoOpenness,catNegativeAffectivity,catDetachment,catAntagonism,catDisinhibition,catPsychoticism
0,198,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,2.541667,0.750000,2.291667,2.583333,3.125000,0.802976,1.440476,1.374107,0.975446,1.15
1,198,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,2.541667,0.750000,2.291667,2.583333,3.125000,0.802976,1.440476,1.374107,0.975446,1.15
2,198,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,2.541667,0.750000,2.291667,2.583333,3.125000,0.802976,1.440476,1.374107,0.975446,1.15
3,198,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,2.541667,0.750000,2.291667,2.583333,3.125000,0.802976,1.440476,1.374107,0.975446,1.15
4,198,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,2.541667,0.750000,2.291667,2.583333,3.125000,0.802976,1.440476,1.374107,0.975446,1.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,408,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,1.625000,2.333333,1.791667,3.083333,1.708333,1.383929,2.065476,0.829018,2.222470,0.35
1251,408,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,1.625000,2.333333,1.791667,3.083333,1.708333,1.383929,2.065476,0.829018,2.222470,0.35
1252,408,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,1.625000,2.333333,1.791667,3.083333,1.708333,1.383929,2.065476,0.829018,2.222470,0.35
1253,408,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,1.625000,2.333333,1.791667,3.083333,1.708333,1.383929,2.065476,0.829018,2.222470,0.35


In [ ]:
ml_model_name = 'lasso'
ls_results_lasso = list()
for pers_name in pers_names:
    ls_results_loocv_lasso, ls_feats_loocv_lasso = loocv_day(df_ml_input_new, pers_name, ml_model_name)
    ls_results_lasso.extend(ls_results_loocv_lasso)
    df_results_lasso_raw = pd.DataFrame(ls_results_loocv_lasso, columns = ['Personality', 'Participant ID', 'MI_iteration', 'Y test', 'Y pred'])
#     df_feats_lasso_raw = pd.DataFrame()
#     df_feats_lasso_raw['Personality'] = df_results_lasso_raw['Personality'].tolist()
#     df_feats_lasso_raw['Participant ID'] = df_results_lasso_raw['Participant ID'].tolist()
#     df_feats_lasso_raw['Feats'] = ls_feats_loocv_lasso
    df_results_lasso_raw.to_csv('lasso_' + pers_name + '_results_raw.csv', index=None)
#     df_feats_lasso_raw.to_csv('lasso_' + pers_name + '_feats_raw.csv', index=None)

In [ ]:
df_results_lasso = pd.DataFrame(ls_results_lasso, columns = ['Personality', 'Participant ID', 'MI_iteration', 'Y test', 'Y pred'])
df_results_lasso.head()

In [ ]:
ls_evalutation_lasso = list()
df_results_lasso_groups = df_results_lasso.groupby('Personality')
df_results_lasso_group_names = df_results_lasso_groups.size().index
for df_results_lasso_group_name in df_results_lasso_group_names:
    ls_evaluation_row_lasso = [df_results_lasso_group_name]
    df_results_lasso_group = df_results_lasso_groups.get_group(df_results_lasso_group_name)
    ls_test_vals_lasso = list()
    ls_pred_vals_lasso = list()
    for val in df_results_lasso_group['Y test'].tolist():
        if len(val) < 1:
            print('Error')
        
        ls_test_vals_lasso.extend(val)

    for val in df_results_lasso_group['Y pred'].tolist():
        if len(val) < 1:
            print('Error')
            
        ls_pred_vals_lasso.extend(val)
    
    df_evaluation_input_lasso = pd.DataFrame()
    df_evaluation_input_lasso['Y_test'] = ls_test_vals_lasso
    df_evaluation_input_lasso['Y_pred'] = ls_pred_vals_lasso
    
#     df_evalution_output_xgb = evaluate(df_evaluation_input_xgb)
    ls_evalutation_lasso.append([df_results_lasso_group_name, 
                               MAE(ls_pred_vals_lasso, ls_test_vals_lasso),
                               MSE(ls_pred_vals_lasso, ls_test_vals_lasso),
                               RMSE(ls_pred_vals_lasso, ls_test_vals_lasso),
                               MAPE(ls_pred_vals_lasso, ls_test_vals_lasso),
                               R2(ls_pred_vals_lasso, ls_test_vals_lasso)])

In [ ]:
df_results_summary = pd.DataFrame(ls_evalutation_lasso, columns = ['Personality', 'MAE', 'MSE', 'RMSE', 'MAPE(%)', 'R2'])
df_results_summary.to_csv('lasso_results_summary.csv', index=None)